## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

c:\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.1.0) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [14]:
tf.__version__

'2.15.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [16]:
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [48]:
messages.shape

(18285, 4)

In [20]:
#messages.reset_index(inplace=True)

In [21]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [63]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []

In [62]:
messages.dropna()
messages.shape

(18285, 4)

In [64]:

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', str(messages['title'].iloc[i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [65]:

len(corpus)

18285

In [66]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [67]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4542, 3763, 1737, 2814, 2112, 2191, 4695, 903, 14, 2995],
 [103, 1680, 2677, 373, 238, 3771, 3640],
 [2391, 1568, 1125, 2914],
 [3194, 3434, 660, 2117, 4345, 2817],
 [4006, 238, 4687, 4981, 2455, 2372, 238, 99, 3213, 3808],
 [792,
  437,
  3991,
  2201,
  1015,
  2797,
  4592,
  4745,
  1671,
  3570,
  3267,
  4602,
  2516,
  690,
  3640],
 [4187, 2302, 3308, 1559, 3952, 1931, 3089, 2030, 4919, 1221, 195],
 [3580, 1057, 1208, 2371, 3284, 53, 2797, 2226, 4919, 1221, 195],
 [115, 154, 413, 2949, 2757, 4179, 4808, 792, 2797, 21],
 [1052, 4184, 2178, 744, 1229, 2166, 241, 2456],
 [2983, 2281, 4920, 4557, 32, 3126, 1930, 2267, 3926, 3044, 3399],
 [2117, 1702, 2112, 4179, 2797, 3284],
 [729, 3017, 719, 3123, 571, 3563, 4497, 1249, 59],
 [456, 1341, 2500, 2377, 3294, 2903, 1789, 4919, 1221, 195],
 [4305, 1700, 1509, 2256, 4512, 4919, 1221, 195],
 [3841, 1376, 4588, 831, 548, 1053, 3040, 3625, 4720, 4688],
 [4207, 1550, 1680],
 [839, 1472, 3682, 1856, 2797, 1925, 4507, 3640],
 [3354, 3374, 2

In [68]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [69]:
onehot_repr[1]

[103, 1680, 2677, 373, 238, 3771, 3640]

### Embedding Representation

In [70]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4542 3763 1737 ...    0    0    0]
 [ 103 1680 2677 ...    0    0    0]
 [2391 1568 1125 ...    0    0    0]
 ...
 [1657 1297 1631 ...    0    0    0]
 [3666 3284 4116 ...    0    0    0]
 [1909 4610 2276 ...    0    0    0]]


In [71]:
embedded_docs[1]

array([ 103, 1680, 2677,  373,  238, 3771, 3640,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [72]:
embedded_docs[0]

array([4542, 3763, 1737, 2814, 2112, 2191, 4695,  903,   14, 2995,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [81]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [74]:
len(embedded_docs),y.shape

(18285, (18285,))

In [82]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [83]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [85]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 30ms/step - loss: 0.3251 - accuracy: 0.8398 - val_loss: 0.2091 - val_accuracy: 0.9117
Epoch 2/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1506 - accuracy: 0.9435 - val_loss: 0.2444 - val_accuracy: 0.9102
Epoch 3/10
192/192 [==============================] - 5s 27ms/step - loss: 0.1098 - accuracy: 0.9597 - val_loss: 0.2345 - val_accuracy: 0.9145
Epoch 4/10
192/192 [==============================] - 5s 26ms/step - loss: 0.0776 - accuracy: 0.9718 - val_loss: 0.2553 - val_accuracy: 0.9122
Epoch 5/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0602 - accuracy: 0.9798 - val_loss: 0.3211 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0532 - accuracy: 0.9811 - val_loss: 0.3490 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0431 - accuracy: 0.9850 - val_loss: 0.3785 - val_accuracy: 0.908

### Performance Metrics And Accuracy

In [86]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 8ms/step


In [87]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [88]:
from sklearn.metrics import confusion_matrix

In [89]:
confusion_matrix(y_test,y_pred)

array([[3147,  272],
       [ 285, 2331]], dtype=int64)

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.907705053852527

In [91]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

